In [59]:
#IMPORTANDO AS BIBLIOTECAS
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.common.exceptions import WebDriverException
from tqdm import tqdm
import sqlite3
import json
import os

In [60]:
#Congiruando o driver
options = Options()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
options.add_argument('--no-sandbox')
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.109 Safari/537.36 OPR/84.0.4316.52")
options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [61]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)

C:\Users\pedro\AppData\Local\Temp\ipykernel_6644\3380238038.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)


In [62]:
#CRIANDO A LISTA DE VALORES
Urls_Extra = []
Sellers_Extra = []
Country_Extra = []
Price_Extra = []
SKU_Extra = []
Title_Extra = []
Installment_Extra_quantidade = []
Installment_Extra_valor_parcela = []
Installment_Extra_valor_total = []
Prox_pag = []

In [63]:
#Função para criar os links de busca
def getting_n_creating_viavarejo_urls(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "%2B")
    df['Product_Name'] = df['Product_Name'].str.replace("-", "%2B")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "+" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://prd-api-partner.viavarejo.com.br/api/search?resultsPerPage=20&terms=" + df['Brand'][0] + "%2B" + df['Product_Name'] + "&page=1&apiKey=extra&"

    return df

In [64]:
Df_Raw = getting_n_creating_viavarejo_urls("Wacom")
Df_Raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom+ctl472,https://prd-api-partner.viavarejo.com.br/api/s...
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom+ctl4100,https://prd-api-partner.viavarejo.com.br/api/s...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wle0,https://prd-api-partner.viavarejo.com.br/api/s...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl4100wlk0,https://prd-api-partner.viavarejo.com.br/api/s...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wlk0,https://prd-api-partner.viavarejo.com.br/api/s...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom+dtc133w0a,https://prd-api-partner.viavarejo.com.br/api/s...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom+pth460,https://prd-api-partner.viavarejo.com.br/api/s...
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom+dtk1660k0a,https://prd-api-partner.viavarejo.com.br/api/s...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom+pth660,https://prd-api-partner.viavarejo.com.br/api/s...
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom+ctl6100wle0,https://prd-api-partner.viavarejo.com.br/api/s...


In [65]:
def get_attributes(url):
    time.sleep(5)
    driver.get(url)
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')

    elemento = soup.find('body')

    dic = json.loads(elemento.text)

    # ID
    number_id = 0
    for i in dic['products']:
        try:
            SKU_Extra.append(dic['products'][number_id]['url'])
        except:
            SKU_Extra.append("ERRO")

        number_id = number_id + 1

    # NOME
    number_name = 0
    for i in dic['products']:
        try:
            Title_Extra.append(dic['products'][number_name]['name'])
        except:
            Title_Extra.append("ERRO")

        number_name = number_name + 1

    # SELLER NUMBER
    number_seller = 0
    for i in dic['products']:
        try:
            Sellers_Extra.append(dic['products'][number_seller]['details']['IdLojista'])
        except:
            Sellers_Extra.append("ERRO")

        number_seller = number_seller + 1

    # INTERNACIONAL
    number_internacional = 0
    for i in dic['products']:
        try:
            Country_Extra.append(dic['products'][number_internacional]['details']['categoryName'])
        except:
            Country_Extra.append("ERRO")

        number_internacional = number_internacional + 1

    # PRICE
    number_price = 0
    for i in dic['products']:
        try:
            Price_Extra.append(dic['products'][number_price]['price'])
        except:
            Price_Extra.append("ERRO")

        number_price = number_price + 1

    # INSTALLMENT PARCELA
    number_parcela = 0
    for i in dic['products']:
        try:
            Installment_Extra_quantidade.append(dic['products'][number_parcela]['installment']['count'])
        except:
            Installment_Extra_quantidade.append("ERRO")

        number_parcela = number_parcela + 1

    # INSTALLMENT VALOR PARCELA
    number_valor = 0
    for i in dic['products']:
        try:
            Installment_Extra_valor_parcela.append(dic['products'][number_valor]['installment']['price'])
        except:
            Installment_Extra_valor_parcela.append("ERRO")

        number_valor = number_valor + 1

    # try da próxima página
    try:
        Prox_pag.append(dic['pagination']['next'])
    except:
        pass

In [66]:
for url in tqdm(Df_Raw['Urls_search']):
    get_attributes(url)

100%|██████████| 10/10 [00:54<00:00,  5.47s/it]


In [67]:
def creating_dataframe(Sellers,Price,Quantidade,SKU,Title,Parcela):
    Dataframe = pd.DataFrame()

    #Colocando Url
    Dataframe['URL'] = SKU

    #Colocando Data
    Dataframe['DATE'] = pd.to_datetime('today', errors='ignore').date()

    Dataframe['MARKETPLACE'] = 'EXTRA'

    correct_seller = []
    #Arrumando sellers
    for seller in Sellers_Extra:
        correct_seller.append(seller[0])

    Dataframe['SELLER'] = correct_seller

    Dataframe['PRICE'] = Price

    Dataframe['PARCEL'] = Quantidade
    Dataframe['INSTALLMENT'] = Parcela

    #Criando coluna de Installment_payment
    Dataframe['INSTALLMENT_PAYMENT'] = Dataframe['PARCEL'] * Dataframe['INSTALLMENT']

    Dataframe['ID'] = Dataframe['URL'].str.partition("IdSku=")[2]
    Dataframe['PRODUCT'] = Title

    #Colocando em ordem
    Dataframe = Dataframe[['DATE','URL','MARKETPLACE','SELLER','PRICE','PARCEL','INSTALLMENT','INSTALLMENT_PAYMENT','ID','PRODUCT']]

    return Dataframe

In [68]:
dataset_viavarejo = creating_dataframe(Sellers_Extra,Price_Extra,Installment_Extra_quantidade,SKU_Extra,Title_Extra,Installment_Extra_valor_parcela)

In [69]:
dataset_viavarejo

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT
0,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,36189,449.00,4,112.25,449.00,1533946489,Mesa Digitalizadora Wacom CTL472
1,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,94202,399.95,4,99.99,399.96,1527638536,Mesa Digitalizadora One By CTL472 - Wacom
2,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,104480,599.90,6,99.98,599.88,1526206918,Mesa Digitalizadora One By Wacom Ctl472
3,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,12231,617.90,6,102.98,617.88,1515816715,Mesa Digitalizadora One By Wacom Ctl472
4,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,11901,476.28,4,119.07,476.28,1506135278,Mesa Digitalizadora One by Wacom CTL472 Pequena
...,...,...,...,...,...,...,...,...,...,...
83,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,E,4147.58,10,414.76,4147.60,1522307841,Mesa Digitalizadora Wacom Pth660 Intuos Pro Media
84,2022-05-31,https://www.extra.com.br/Esporte-Lazer/jogosde...,EXTRA,E,6599.00,3,2199.67,6599.01,1511666432,Mesa Digitalizadora Wacom Intuos Pro Média PTH660
85,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,E,4408.92,10,440.89,4408.90,1522307786,Mesa Digitalizadora Wacom Pth660p Intuos Pro M...
86,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,E,3669.90,10,366.99,3669.90,12778347,Mesa Digitalizadora Wacom Intuos Pro Paper Edi...


In [125]:
def get_sellers(url,sellers):
    # driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)

    #Criando o dataset para retirar as duplicadas
    df_sellers = pd.DataFrame()

    df_sellers['URL'] = url
    df_sellers['SELLER'] = sellers

    #Retirando as duplicadas
    df_sellers.drop_duplicates(subset=['SELLER'], inplace=True)

    # sellers_name  =[]
    #
    # for url in tqdm(df_sellers['URL']):
    #     driver.get(url)
    #     time.sleep(20)
    #
    #     html = driver.page_source
    #
    #     Soup = BeautifulSoup(html, 'html.parser')
    #
    #     try:
    #         sellers_name.append(Soup.find(class_='e1vg858b0 css-1kq6ah1 e1g7zzz30').text)
    #     except:
    #         sellers_name.append("ERRO")
    #
    # df_sellers['NAMES'] = sellers_name

    return df_sellers

In [127]:
sellers_pd = get_sellers(dataset_viavarejo['URL'], dataset_viavarejo['SELLER'])
sellers_pd

,URL,SELLER
0,https://www.extra.com.br/acessorios-inovacoes/...,36189
1,https://www.extra.com.br/acessorios-inovacoes/...,94202
2,https://www.extra.com.br/acessorios-inovacoes/...,104480
3,https://www.extra.com.br/acessorios-inovacoes/...,12231
4,https://www.extra.com.br/acessorios-inovacoes/...,11901
5,https://www.extra.com.br/acessorios-inovacoes/...,44309
6,https://www.extra.com.br/acessorios-inovacoes/...,16189
8,https://www.extra.com.br/acessorios-inovacoes/...,102177
9,https://www.extra.com.br/acessorios-inovacoes/...,81620
10,https://www.extra.com.br/acessorios-inovacoes/...,14201


In [109]:
sellers_name = []
internacional_list = []
for url in sellers_pd['URL']:
    driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)
    driver.get(url)
    time.sleep(5)

    html = driver.page_source

    driver.close()

    Soup = BeautifulSoup(html, 'html.parser')

    try:
        sellers_name.append(Soup.find(class_='e1vg858b0 css-1kq6ah1 e1g7zzz30').text)
        try:
            internacional_list.append(Soup.find('img', attrs={'data-testid':'stamps'})['alt'])
        except:
            internacional_list.append("NACIONAL")
    except:
        sellers_name.append("ERRO")
        internacional_list.append("ERRO")

C:\Users\pedro\AppData\Local\Temp\ipykernel_6644\3961321297.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)


In [115]:
sellers_pd['Name'] = sellers_name
sellers_pd['INTERNACIONAL'] = internacional_list

#Mudando o nome do seller
sellers_pd['Name'] = sellers_pd['Name'].str.partition("por")[2]

sellers_pd

,URL,SELLER,Name,INTERNACIONAL
0,https://www.extra.com.br/acessorios-inovacoes/...,36189,BRDF Informatica,NACIONAL
1,https://www.extra.com.br/acessorios-inovacoes/...,94202,Info Store,NACIONAL
2,https://www.extra.com.br/acessorios-inovacoes/...,104480,URBAN GATE,NACIONAL
3,https://www.extra.com.br/acessorios-inovacoes/...,12231,Olist,NACIONAL
4,https://www.extra.com.br/acessorios-inovacoes/...,11901,,ERRO
5,https://www.extra.com.br/acessorios-inovacoes/...,44309,e-spot,NACIONAL
6,https://www.extra.com.br/acessorios-inovacoes/...,16189,Qualitech Informática,NACIONAL
8,https://www.extra.com.br/acessorios-inovacoes/...,102177,SYMA SOLUTIONS,NACIONAL
9,https://www.extra.com.br/acessorios-inovacoes/...,81620,VSP,NACIONAL
10,https://www.extra.com.br/acessorios-inovacoes/...,14201,Oficina dos Bits,NACIONAL


In [121]:
sellers_name_correct = []

for code in dataset_viavarejo['SELLER']:
    sellers_name_correct.append(sellers_pd.loc[sellers_pd['SELLER'] == code,'Name'].values[0])

internacional_name_correct = []

for code in dataset_viavarejo['SELLER']:
    internacional_name_correct.append(sellers_pd.loc[sellers_pd['SELLER'] == code,'INTERNACIONAL'].values[0])

In [117]:
sellers_pd.loc[sellers_pd['SELLER'] == '36189', 'Name'].values[0]

' BRDF Informatica'

In [122]:
dataset_viavarejo['Seller_Name'] = sellers_name_correct
dataset_viavarejo['INTERNACIONAL'] = internacional_name_correct
dataset_viavarejo

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,INSTALLMENT_PAYMENT,ID,PRODUCT,Seller_Name,INTERNACIONAL
0,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,36189,449.00,4,112.25,449.00,1533946489,Mesa Digitalizadora Wacom CTL472,BRDF Informatica,NACIONAL
1,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,94202,399.95,4,99.99,399.96,1527638536,Mesa Digitalizadora One By CTL472 - Wacom,Info Store,NACIONAL
2,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,104480,599.90,6,99.98,599.88,1526206918,Mesa Digitalizadora One By Wacom Ctl472,URBAN GATE,NACIONAL
3,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,12231,617.90,6,102.98,617.88,1515816715,Mesa Digitalizadora One By Wacom Ctl472,Olist,NACIONAL
4,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,11901,476.28,4,119.07,476.28,1506135278,Mesa Digitalizadora One by Wacom CTL472 Pequena,,ERRO
...,...,...,...,...,...,...,...,...,...,...,...,...
83,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,E,4147.58,10,414.76,4147.60,1522307841,Mesa Digitalizadora Wacom Pth660 Intuos Pro Media,,ERRO
84,2022-05-31,https://www.extra.com.br/Esporte-Lazer/jogosde...,EXTRA,E,6599.00,3,2199.67,6599.01,1511666432,Mesa Digitalizadora Wacom Intuos Pro Média PTH660,,ERRO
85,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,E,4408.92,10,440.89,4408.90,1522307786,Mesa Digitalizadora Wacom Pth660p Intuos Pro M...,,ERRO
86,2022-05-31,https://www.extra.com.br/acessorios-inovacoes/...,EXTRA,E,3669.90,10,366.99,3669.90,12778347,Mesa Digitalizadora Wacom Intuos Pro Paper Edi...,,ERRO


In [105]:
driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)
driver.get('https://www.extra.com.br/Tablets/Tablet/TabletAndroid/wacom-ctl472k1a-um-por-wacom-graphic-drawing-tablet-tablet-para-iniciantes-pequeno-1515626407.html?IdSku=1515626407')
time.sleep(10)

html_str = driver.page_source

Soup = BeautifulSoup(html_str, 'html.parser')

C:\Users\pedro\AppData\Local\Temp\ipykernel_6644\4100146394.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Selenium\Selenium_101", options=options)


In [106]:
print(Soup.find(class_='eq9v2q60 css-1fo2kvz eym5xli0'))

None


In [108]:
Soup.find('img', attrs={'data-testid':'stamps'})['alt']

'AC - Selo Produtos Importados NocNoc'